In [31]:
# Imports e paths

from pathlib import Path
import pandas as pd
from gensim.corpora import Dictionary, MmCorpus

BASE = Path("../../data")
RAW = BASE / "raw"
INTERIM_LDA = BASE / "interim" / "lda"
PROCESSED_LDA = BASE / "processed" / "lda"

PREP_CSV = INTERIM_LDA / "prep.csv"

DICT_PATH = INTERIM_LDA / "vocab_bow.dict"
BOW_MM_PATH = INTERIM_LDA / "bow.mm"
VOCAB_TERMS_CSV = INTERIM_LDA / "vocab_terms.csv"
BOW_INDEX_CSV = INTERIM_LDA / "bow_index.csv"

for p in [INTERIM_LDA]:
    p.mkdir(parents=True, exist_ok=True)

print("Usando base:", BASE.resolve())
print("Arquivos de saída serão gravados em:", INTERIM_LDA.resolve())


Usando base: C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data
Arquivos de saída serão gravados em: C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data\interim\lda


In [32]:
# Leitura (prep.csv) e sanity checks

assert PREP_CSV.exists(), f"Arquivo não encontrado: {PREP_CSV}"

df = pd.read_csv(PREP_CSV)
cols_ok = {"DOC_ID", "RESUMO_PREP"}
missing = cols_ok - set(df.columns)
assert not missing, f"Colunas ausentes no prep.csv: {missing}"

total_docs = len(df)
print(f"prep.csv carregado. Linhas totais = {total_docs}")
print("Colunas:", list(df.columns))


prep.csv carregado. Linhas totais = 423
Colunas: ['DOC_ID', 'ano', 'titulo', 'autor', 'orientador', 'resumo', 'url', 'RESUMO_PREP']


In [33]:
# Seleção do subconjunto válido de trabalho e tokenização

if "TOKENS_LEN" in df.columns:
    df_work = df[df["TOKENS_LEN"] > 0].copy()
    criterio = 'TOKENS_LEN > 0'
else:
    df_work = df[df["RESUMO_PREP"].astype(str).str.strip().ne("")].copy()
    criterio = 'RESUMO_PREP != "" (strip)'

assert "DOC_ID" in df_work.columns, "DOC_ID ausente no prep.csv"
df_work = df_work.sort_values("DOC_ID").reset_index(drop=True)

docs_total = len(df_work)
print(f"Critério de seleção: {criterio}")
print(f"Docs usados (após seleção) = {docs_total} / {total_docs} (totais)")

df_work["__TOKENS__"] = df_work["RESUMO_PREP"].astype(str).str.split()

print("Primeiros 3 DOC_ID após ordenação:", df_work["DOC_ID"].head(3).tolist())
print("Exemplo de tokens:", df_work["__TOKENS__"].iloc[0][:15] if docs_total > 0 else [])

Critério de seleção: RESUMO_PREP != "" (strip)
Docs usados (após seleção) = 423 / 423 (totais)
Primeiros 3 DOC_ID após ordenação: [0, 1, 2]
Exemplo de tokens: ['gerenciar', 'agenda', 'fornecer_informacao', 'cliente', 'tarefa', 'oner', 'pequeno', 'empreendedor', 'caso', 'gerencia', 'operar', 'negocio', 'sozinho', 'precisar', 'maximizar']


In [34]:
# Vocabulário e filtragem por frequência

docs_list = df_work["__TOKENS__"].tolist()
doc_ids = df_work["DOC_ID"].tolist()
n_docs_for_df = len(docs_list)

dictionary = Dictionary(docs_list)
vocab_init = len(dictionary)
print(f"|V| inicial (antes do filtro): {vocab_init}")

dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=None)
dictionary.compactify()
vocab_final = len(dictionary)
print(f"|V| após filter_extremes: {vocab_final}")

if vocab_final == 0:
    raise RuntimeError(
        "Após o filtro de frequência, o vocabulário ficou vazio. "
        "Revise o 01_prep ou os limiares no_below/no_above para este corpus."
    )

dfs = dictionary.dfs
if len(dfs) == 0:
    raise RuntimeError("dictionary.dfs vazio após filtro — algo inconsistente com o vocabulário.")

min_df = min(dfs.values())
max_df = max(dfs.values())
max_df_expected = int(n_docs_for_df * 0.5)

print(f"DF pós-filtro: min={min_df} | max={max_df} | limite teórico max ≤ {max_df_expected}")

assert min_df >= 5, f"min_df={min_df} < 5 — inesperado após no_below=5"
assert max_df <= max_df_expected, (
    f"max_df={max_df} > {max_df_expected} — inesperado após no_above=0.5 "
    f"(n_docs_considerados={n_docs_for_df})"
)

|V| inicial (antes do filtro): 4968
|V| após filter_extremes: 1218
DF pós-filtro: min=5 | max=186 | limite teórico max ≤ 211


In [35]:
# Construção do corpus BoW e remoção de docs vazios

import pandas as pd
import numpy as np

# Construção de BoW para todos e filtrar vazios após filtro de vocabulário
corpus_bow_all = [dictionary.doc2bow(doc) for doc in docs_list]
non_empty_mask = [len(b) > 0 for b in corpus_bow_all]

n_nonempty = sum(non_empty_mask)
n_empty = len(non_empty_mask) - n_nonempty
if n_nonempty == 0:
    raise RuntimeError("Todos os documentos ficaram vazios após a filtragem do vocabulário.")

corpus_bow = [b for b, keep in zip(corpus_bow_all, non_empty_mask) if keep]
doc_ids_used = [d for d, keep in zip(doc_ids, non_empty_mask) if keep]

# Métricas de sparsidade
nnz = sum(len(b) for b in corpus_bow)
n_docs_used = len(corpus_bow)
vocab_final = len(dictionary)  # reuso
sparsity = nnz / (n_docs_used * vocab_final) if vocab_final > 0 else float("nan")

print(f"Docs com BoW != vazio: {n_docs_used} (removidos por ficarem vazios: {n_empty})")
print(f"nnz = {nnz} | sparsidade = {sparsity:.6f}")

# Salvamentos básicos: dicionário, Matrix Market, vocab_terms.csv, bow_index.csv
# Dicionário
dictionary.save(str(DICT_PATH))

# MmCorpus
MmCorpus.serialize(str(BOW_MM_PATH), corpus_bow)

# vocab_terms.csv
dfs = dictionary.dfs
records = [(token_id, dictionary[token_id], dfreq) for token_id, dfreq in dfs.items()]
vocab_df = pd.DataFrame(records, columns=["token_id", "token", "df"])\
            .sort_values(by=["df", "token"], ascending=[False, True])
vocab_df.to_csv(VOCAB_TERMS_CSV, index=False, encoding="utf-8")

# bow_index.csv (row_idx <-> DOC_ID)
bow_index_df = pd.DataFrame({"row_idx": range(n_docs_used), "DOC_ID": doc_ids_used})
bow_index_df.to_csv(BOW_INDEX_CSV, index=False, encoding="utf-8")

print("Arquivos gravados:")
print(" -", DICT_PATH.name)
print(" -", BOW_MM_PATH.name)
print(" -", VOCAB_TERMS_CSV.name)
print(" -", BOW_INDEX_CSV.name)

# Checagens do bow_index: monotonicidade e abrangência
# Monotonicidade estrita em DOC_ID
is_monotonic_strict = bow_index_df["DOC_ID"].is_monotonic_increasing and bow_index_df["DOC_ID"].is_unique
assert is_monotonic_strict, (
    "DOC_ID em bow_index não está estritamente crescente/único — verifique ordenação e filtragem."
)

# Abrangência: todos os DOC_ID usados estão contidos nos DOC_ID selecionados originalmente (pós-ordenação)
set_used = set(doc_ids_used)
set_selected = set(doc_ids)
assert set_used.issubset(set_selected), "Existem DOC_ID em bow_index que não pertencem ao conjunto selecionado."
assert len(set_used) == n_docs_used, "Tamanho do conjunto de DOC_ID usados difere de n_docs_used."

print("bow_index: checagens OK (monotonicidade estrita e abrangência confirmadas).")

# Exportar CSR: bow_csr.npz para diagnósticos auxiliares
try:
    from scipy.sparse import csr_matrix, save_npz

    if vocab_final > 0 and n_docs_used > 0:
        data = []
        rows = []
        cols = []
        for r, doc in enumerate(corpus_bow):
            if doc:  # lista de (term_id, count)
                c_idx, c_val = zip(*doc)
                rows.extend([r] * len(c_idx))
                cols.extend(c_idx)
                data.extend(c_val)
        csr = csr_matrix((data, (rows, cols)), shape=(n_docs_used, vocab_final), dtype=np.float64)

        CSR_PATH = INTERIM_LDA / "bow_csr.npz"
        save_npz(str(CSR_PATH), csr)
        # Validação rápida
        assert csr.nnz == nnz, f"CSR nnz ({csr.nnz}) != nnz do corpus_bow ({nnz})"
        print(f"CSR salvo em: {CSR_PATH.name} (shape={csr.shape}, nnz={csr.nnz}, densidade={csr.nnz/(csr.shape[0]*csr.shape[1]):.6f})")
    else:
        print("CSR não gerado (vocab_final ou n_docs_used = 0).")
except ImportError:
    print("scipy.sparse não disponível — pulando exportação de bow_csr.npz (opcional).")

Docs com BoW != vazio: 423 (removidos por ficarem vazios: 0)
nnz = 20066 | sparsidade = 0.038947
Arquivos gravados:
 - vocab_bow.dict
 - bow.mm
 - vocab_terms.csv
 - bow_index.csv
bow_index: checagens OK (monotonicidade estrita e abrangência confirmadas).
CSR salvo em: bow_csr.npz (shape=(423, 1218), nnz=20066, densidade=0.038947)


In [36]:
# Relatório rápido
import json

# Top 15 termos por DF
print("\nTop 15 termos por DF:")
display(vocab_df.head(15)[["token", "df"]])

# Manifest de auditoria para próxima etapa (LDA)
manifest = {
    "docs_totais": int(total_docs),
    "docs_usados": int(n_docs_used),
    "vocab_inicial": int(vocab_init),
    "vocab_final": int(vocab_final),
    "nnz": int(nnz),
    "sparsidade": float(sparsity),
    "no_below": 5,
    "no_above": 0.5,
    "df_min": int(vocab_df["df"].min()),
    "df_max": int(vocab_df["df"].max()),
    "bow_shape": [int(n_docs_used), int(vocab_final)],
    "artefatos": {
        "dictionary": str(DICT_PATH),
        "mm_corpus": str(BOW_MM_PATH),
        "vocab_terms": str(VOCAB_TERMS_CSV),
        "bow_index": str(BOW_INDEX_CSV),
        "csr_npz": str(INTERIM_LDA / "bow_csr.npz")
    }
}
with open(INTERIM_LDA / "bow_manifest.json", "w", encoding="utf-8") as f:
    json.dump(manifest, f, ensure_ascii=False, indent=2)

print("\nManifest salvo em:", (INTERIM_LDA / "bow_manifest.json").resolve())
print(json.dumps(manifest, ensure_ascii=False, indent=2))



Top 15 termos por DF:


,token,df
65,utilizar,186
159,trabalho,180
126,objetivo,142
141,dado,138
131,uso,134
35,usuario,121
81,estudo,117
58,ferramenta,113
11,resultado,110
55,contexto,109



Manifest salvo em: C:\Users\User\Desktop\TCC\Notebooks locais\analise_topicos_tcc\data\interim\lda\bow_manifest.json
{
  "docs_totais": 423,
  "docs_usados": 423,
  "vocab_inicial": 4968,
  "vocab_final": 1218,
  "nnz": 20066,
  "sparsidade": 0.038946923026160006,
  "no_below": 5,
  "no_above": 0.5,
  "df_min": 5,
  "df_max": 186,
  "bow_shape": [
    423,
    1218
  ],
  "artefatos": {
    "dictionary": "..\\..\\data\\interim\\lda\\vocab_bow.dict",
    "mm_corpus": "..\\..\\data\\interim\\lda\\bow.mm",
    "vocab_terms": "..\\..\\data\\interim\\lda\\vocab_terms.csv",
    "bow_index": "..\\..\\data\\interim\\lda\\bow_index.csv",
    "csr_npz": "..\\..\\data\\interim\\lda\\bow_csr.npz"
  }
}
